In [1]:
import pandas as pd
import re
import numpy as np

In [2]:
job = pd.read_csv("dataset1_with_error.csv",encoding = "utf-8_sig")

In [3]:
print(job.shape)
job.head()

(25077, 11)


,Id,Title,Location,ContractType,ContractTime,Company,Category,Salary per annum,SourceName,OpenDate,CloseDate
0,12612628,Engineering Systems Analyst,Dorking,not available,permanent,Gregory Martin International,Engineering Jobs,24996,cv-library.co.uk,20121103T000000,20121203T000000
1,12612830,Stress Engineer Glasgow,Glasgow,not available,permanent,Gregory Martin International,Engineering Jobs,30000,cv-library.co.uk,20130108T150000,20130408T150000
2,12612844,Modelling and simulation analyst,Hampshire,not available,permanent,Gregory Martin International,Engineering Jobs,30000,cv-library.co.uk,20130726T150000,20130924T150000
3,12613049,Engineering Systems Analyst / Mathematical Mod...,Surrey,not available,permanent,Gregory Martin International,Engineering Jobs,27504,cv-library.co.uk,20121214T000000,20130314T000000
4,12613647,"Pioneer, Miser Engineering Systems Analyst",Surrey,not available,permanent,Gregory Martin International,Engineering Jobs,24996,cv-library.co.uk,20131025T000000,20131224T000000


In [4]:
job.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25077 entries, 0 to 25076
Data columns (total 11 columns):
Id                  25077 non-null int64
Title               25077 non-null object
Location            25077 non-null object
ContractType        25077 non-null object
ContractTime        25077 non-null object
Company             21242 non-null object
Category            25077 non-null object
Salary per annum    25077 non-null object
SourceName          25077 non-null object
OpenDate            25077 non-null object
CloseDate           25077 non-null object
dtypes: int64(1), object(10)
memory usage: 2.1+ MB


In [5]:
job.describe(include=['O'])

,Title,Location,ContractType,ContractTime,Company,Category,Salary per annum,SourceName,OpenDate,CloseDate
count,25077,25077,25077,25077,21242,25077,25077,25077,25077,25077
unique,25077,482,3,3,4879,8,1589,90,2203,2400
top,Asset Sales Specialist,UK,not available,permanent,UKStaffsearch,IT Jobs,35004,totaljobs.com,20130127T150000,20130526T120000
freq,1,3996,19499,16194,248,7085,1011,5335,27,23


In [6]:
df = job.copy()

### Id

In [7]:
#To check that each record are in unique "Id"
df[df.duplicated(["Id"], keep=False)]

,Id,Title,Location,ContractType,ContractTime,Company,Category,Salary per annum,SourceName,OpenDate,CloseDate


### Irregularities

There are different errors in the Title column. 
1. It contains "*" symbols in some places. But this symbol cannot be replace as not enough related information can be used. If I delete the symbols, some information may be lost.
2. The information in this column is irregular. Some of the titles even contain location information. 
3. It can be fixed by regularize the information to be the same fornt format.

### Title

In [8]:
#To capitalize the first character of each word only, which make the information in the same format.
df["Title"] = df["Title"].str.title()
df["Title"].head(10)

0                          Engineering Systems Analyst
1                              Stress Engineer Glasgow
2                     Modelling And Simulation Analyst
3    Engineering Systems Analyst / Mathematical Mod...
4           Pioneer, Miser Engineering Systems Analyst
5              Trainee Mortgage Advisor  East Midlands
6                     Project Engineer, Pharmaceutical
7    Chef De Partie  Award Winning Restaurant  Exce...
8                                     Quality Engineer
9    Chef De Partie  Award Winning Dining  Live In ...
Name: Title, dtype: object

Following the requirement, change the contract type and contract time from "not available" to "non-specified"

### ContractType

In [9]:
df['ContractType'] = df['ContractType'].str.replace("not available","non-specified")
#df

### ContractTime

In [10]:
df['ContractTime'] = df['ContractTime'].str.replace("not available","non-specified")
#df

### Compay

In the company column, there is a special record that contains the '.' instead of a company name. Although there is some missing values in this column, it can not be fixed as not enough information. This record can be seen as missing value and '.' here may not contain any useful information so delete it. 

In [11]:
# find out the company name that is with '.' only
c = df.iloc[:,5]
reg = re.compile("^\.")
for index,data in enumerate(c.values):
    if reg.search(str(data)) != None:
        print(index,data)

20472 .


In [12]:
df.Company.replace('.',np.nan,inplace = True)

In [13]:
for i, data in enumerate(df.Company):
    if "Limited" in str(data):
        df.iloc[i,5] = df.iloc[i,5][:-7] + 'Ltd'
    if "LIMITED" in str(data):
        df.iloc[i,5] = df.iloc[i,5][:-7] + 'Ltd'
    if "Ltd" in str(data):
        df.iloc[i,5] = df.iloc[i,5][:-3] + 'Ltd'
    if "LTD" in str(data):
        df.iloc[i,5] = df.iloc[i,5][:-3] + 'Ltd'

### Data inconsistency
The feature "Salary per annum" has values with character "k" and range values,which are inconsistent with the other normal values.

### Salary per annum

In [14]:
df["Salary per annum"].iloc[133:137]

133      30K
134    32496
135    25440
136    30000
Name: Salary per annum, dtype: object

In [15]:
#replace "K" with numerical values."K" means Kilo, so replace it with "000"
df["Salary per annum"] = df["Salary per annum"].str.replace("K","000")
df["Salary per annum"].iloc[133:137]

133    30000
134    32496
135    25440
136    30000
Name: Salary per annum, dtype: object

There are still many range values in salary column, here use the mean value to replace the range values.

In [16]:
#The salary column have range values
df["Salary per annum"].iloc[1525:1529]

1525    30871.199999999997 - 34120.8
1526                           56640
1527                           40500
1528                           38004
Name: Salary per annum, dtype: object

In [17]:
s2 = df.iloc[:,7]
#use reglar expression to find the range values
reg = re.compile("-")
# find the index and the data to be splited and count the mean values
for index,data in enumerate(s2.values):
    if reg.search(data) != None:
        low_salary = float(data.split(' ')[0])
        high_salary = float(data.split(' ')[2])
        mean_salary = round(low_salary + high_salary)//2
        df.iloc[index,7] = mean_salary

In [18]:
#The range valuse were changed to be mean values 
df["Salary per annum"].iloc[1525:1529]

1525    32496
1526    56640
1527    40500
1528    38004
Name: Salary per annum, dtype: object

### Source Name

In the source name column, there are some source names that are in incorrect format and need to be fixed.

In [19]:
# find out the source names that are not normal ip address
n = df.iloc[:,8]
reg = re.compile("@")
reg1 = re.compile("\.")
for index,data in enumerate(n.values):
    if reg.search(data) != None:
        print(index,data)
    if reg1.search(data) == None:
        print(index,data)

5663 monashstudent
12871 jobcareer
15379 admin@caterer.com


In [20]:
df.SourceName.replace("monashstudent","www.monash.edu/careers",inplace = True)
df.SourceName.replace("jobcareer","jobcareer.com",inplace = True)
df.SourceName.replace("admin@caterer.com","caterer.com",inplace = True)

In [21]:
df["SourceName"].iloc[5663:5664]

5663    www.monash.edu/careers
Name: SourceName, dtype: object

In [22]:
df["SourceName"].iloc[12871:12872]

12871    jobcareer.com
Name: SourceName, dtype: object

In [23]:
df["SourceName"].iloc[15379:15380]

15379    caterer.com
Name: SourceName, dtype: object

### Violations of the Integrity constraint

The two date columns contains values that are dates over 31 or months over 12. It can assumes that these dates put the date and the months in the wrong places. To find out these data and exchange their places. 

### OpenDate

In [24]:
#To check use a function to check the date
def check_date(t):
    for i,data in enumerate(t):
        month = data.split("T")[0][4:6]
        date = data.split("T")[0][6:8]
        if int(month) > 12:
            print("Check the date in row %s" %i)
        if int(date) > 31:
            print("Check the date in row %s" %i)

In [25]:
#To check the OpenDate column and find out the wrong date
check_date(df["OpenDate"])

Check the date in row 1102
Check the date in row 2104
Check the date in row 2839
Check the date in row 5707
Check the date in row 10881
Check the date in row 11948
Check the date in row 15353
Check the date in row 22918
Check the date in row 23007
Check the date in row 23169


### CloseDate

In [26]:
#To check the CloseDate column but there is not any wrong date
check_date(df["CloseDate"])

In [27]:
#To check use a function to check the time
def check_Time(t):
    for i,data in enumerate(t):
        hour = data.split("T")[1][0:2]
        minute = data.split("T")[1][2:4]
        second = data.split("T")[1][4:6]
        if int(hour) > 60:
            print("Check the date in row %s" %i)
        if int(minute) > 60:
            print("Check the date in row %s" %i)
        if int(second) > 60:
            print("Check the date in row %s" %i)

In [28]:
#To check the OpenDate column but there is not any wrong date
check_Time(df["OpenDate"])

In [29]:
#To check the CloseDate column but there is not any wrong date
check_Time(df["CloseDate"])

To correct the OpenDate and also compare it to the CloseDate. There is some OpenDate that are later than the CloseDate, so it is needed to exchange their places.

In [30]:
t1 = df.iloc[:,9]
t2 = df.iloc[:,10]
for i,data in enumerate(t1):
    month = data.split("T")[0][4:6]
    date = data.split("T")[0][6:8]
    if int(month) > 12:
        month, date = date, month
        new_t1 = month + date
        df.iloc[i,9]= df.iloc[i,9].replace(df.iloc[i,9].split("T")[0][4:8],new_t1)
    if int(df.iloc[i,9].split("T")[0]) > int(df.iloc[i,10].split("T")[0]):
        print("Check row %s" %i)
        df.iloc[i,9],df.iloc[i,10] = df.iloc[i,10],df.iloc[i,9]
        print("Change result:", df.iloc[i,9],df.iloc[i,10])

Check row 6659
Change result: 20130404T000000 20130703T000000
Check row 9568
Change result: 20120205T000000 20120306T000000
Check row 11043
Change result: 20131006T000000 20131105T000000
Check row 12473
Change result: 20131023T150000 20131222T150000
Check row 19142
Change result: 20120505T150000 20120803T150000
Check row 24206
Change result: 20130324T150000 20130622T150000
Check row 24297
Change result: 20121018T000000 20130116T000000
Check row 25039
Change result: 20130608T120000 20130708T120000


In [31]:
#To check the changing result is correct
print(df["OpenDate"].iloc[1102:1103])
print(df["CloseDate"].iloc[1102:1103])
print(df["OpenDate"].iloc[6659:6660])
print(df["CloseDate"].iloc[6659:6660])

1102    20130318T120000
Name: OpenDate, dtype: object
1102    20130417T120000
Name: CloseDate, dtype: object
6659    20130404T000000
Name: OpenDate, dtype: object
6659    20130703T000000
Name: CloseDate, dtype: object


### Lexical errors

### Location

In [32]:
#To check the appreance of each location
df['Location'].value_counts()

UK                      3996
London                  2743
South East London       1433
The City                 558
Leeds                    343
Manchester               334
Surrey                   317
Central London           316
Reading                  300
Birmingham               292
West Midlands            273
East Sheen               222
Bristol                  222
Berkshire                202
Nottingham               196
Oxford                   190
Hampshire                184
Milton Keynes            180
Cambridge                172
Sheffield                160
Kent                     158
Newcastle Upon Tyne      156
Hertfordshire            155
Guildford                147
Aberdeen                 141
Leicester                139
Cheshire                 133
Oxfordshire              127
West Yorkshire           127
Belfast                  125
                        ... 
St. Ives                   4
Surey                      4
Reeding                    4
Cobham        

After checking the location names with less appearance in the column, I found that the these are correct location name in England. So I need to compare the location names with each other in the same column to find out the particular ones that are incorrect.

In [33]:
#To use a matching library to check the similarity of each location name
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

unique_location = df['Location'].unique()
for i, location in enumerate(unique_location):
    for j,location2 in enumerate(unique_location[i+1:]):
        similar_rate = fuzz.ratio(location,location2)
        i += 1
        #To use the similar rate of 85 to find out the dataset that have similar location name
        if similar_rate >= 80 :
            print(i,location,j,location2)

57 Surrey 53 Surey
176 East Midlands 171 West Midlands
185 North East England 172 North West England
354 North East England 341 South East England
33 North Yorkshire 19 South Yorkshire
147 North Yorkshire 133 North Ayrshire
420 Buckinghamshire 404 Buckingham
56 Manchester 34 Winchester
140 Bradford 116 Bedford
142 South Yorkshire 108 South Lanarkshire
55 Nottinghamshire 20 Nottingham
117 Northamptonshire 81 Northampton
149 Hertfordshire 110 Bedfordshire
276 Hertfordshire 237 Herefordshire
169 Warrington 128 Workington
204 Warrington 163 Washington
225 Warrington 184 Darlington
206 South East London 163 South West London
241 Leeds 196 Lewes
453 Leeds 408 Leads
75 North West London 29 North London
206 North West London 160 South West London
93 West Sussex 42 East Sussex
285 Norwich 233 Northwich
479 Nottingham 423 Nottinham
182 Winchester 125 Chichester
117 Southampton 55 Northampton
211 West London 146 East London
135 Boston 69 Bolton
376 Leicester 309 Bicester
112 Harlow 37 Harrow
129 

After searching all the location names of the list above in Google, I find out that some location names are totally not exist in England. They are:
1. Surey
2. Nottinham
3. Oxfords
4. Reeding
5. Leads

So I need to fix this location name to be the correct ones.

In [34]:
for data in (df.Location):
    if data == "Surey":
        df.Location.replace("Surey","Surrey",inplace = True)
    if data == "Nottinham":
        df.Location.replace("Nottinham","Nottingham",inplace = True)
    if data == "Oxfords":
        df.Location.replace("Oxfords","Oxford",inplace = True)
    if data == "Reeding":
        df.Location.replace("Reeding","Reading",inplace = True)
    if data == "Leads":
        df.Location.replace("Leads","Leeds",inplace = True)

In [35]:
df.iloc[122:123]

,Id,Title,Location,ContractType,ContractTime,Company,Category,Salary per annum,SourceName,OpenDate,CloseDate
122,46626802,Nursing Home Staff Nurse Job Surrey,Surrey,full_time,non-specified,NaN,Healthcare & Nursing Jobs,23040,careworx.co.uk,20130512T120000,20130810T120000


### Category 

In the category column, there are 8 categories and each seems to be correct.

In [36]:
df['Category'].value_counts()

IT Jobs                             7085
Healthcare & Nursing Jobs           4334
Engineering Jobs                    3458
Accounting & Finance Jobs           3099
Sales Jobs                          2609
Hospitality & Catering Jobs         2124
Teaching Jobs                       1378
PR, Advertising & Marketing Jobs     990
Name: Category, dtype: int64

In [37]:
df1 = df.set_index('Id', inplace = False)

In [38]:
print(df1.shape)

(25077, 10)


In [39]:
df1.to_csv('dataset1_solution.csv',encoding = "utf-8_sig")  

### Summary

1. To identify the data errors in according to the data characteristics and find the errors column by column. This can be helpful to find out error in more detail.
2. According to the data, some data correction need to combine with assumption. The date column is fixed by exchanging the data places.
3. For categorical values, the correction is more complicated. It is needed to use various packages and methods to fixed the errors.